In [6]:
from load_data import read_data
import tensorflow as tf
import numpy as np

In [7]:
# Load data
X, Y = read_data()
# # y to one-hot
Y = tf.keras.utils.to_categorical(Y, num_classes=10)
print(f"X data shape: {X.shape}")
print(f"y data shape: {Y.shape}")

X data shape: (1000, 222, 3)
y data shape: (1000, 10)


In [8]:
def get_conv_model(samples, features):
    """ take in a padded sequence of 3d coordinates, and output a class label
    """
    inputs = tf.keras.Input(shape=(samples,features))
    x = tf.keras.layers.Conv1D(32, 3, activation='relu')(inputs)
    x = tf.keras.layers.Conv1D(64, 3, activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(3)(x)
    x = tf.keras.layers.Conv1D(64, 3, activation='relu')(x)
    x = tf.keras.layers.Conv1D(128, 3, activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs, x)
    return model

In [9]:
# Train the model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import shutil
from sklearn.preprocessing import StandardScaler, MinMaxScaler

model = get_conv_model(X.shape[1], X.shape[2])

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
#X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

# Train the model
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
shutil.rmtree('./tblogs', ignore_errors=True)
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir='./tblogs')

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stop, tensorboard_cb])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Confusion matrix: \n{confusion_matrix(y_test, y_pred)}")

# Save the model
model.save('model.h5')

X_train shape: (800, 222, 3)
y_train shape: (800, 10)
X_test shape: (200, 222, 3)
Epoch 1/20
20/20 [==============================] - 1s 11ms/step - loss: 2.9871 - accuracy: 0.1187 - val_loss: 2.2724 - val_accuracy: 0.2062
Epoch 2/20
20/20 [==============================] - 0s 6ms/step - loss: 2.2339 - accuracy: 0.1859 - val_loss: 2.1945 - val_accuracy: 0.1437
Epoch 3/20
20/20 [==============================] - 0s 6ms/step - loss: 2.0226 - accuracy: 0.2719 - val_loss: 2.0814 - val_accuracy: 0.2125
Epoch 4/20
20/20 [==============================] - 0s 6ms/step - loss: 1.8156 - accuracy: 0.3469 - val_loss: 1.8890 - val_accuracy: 0.3187
Epoch 5/20
20/20 [==============================] - 0s 6ms/step - loss: 1.5755 - accuracy: 0.4469 - val_loss: 1.6478 - val_accuracy: 0.4250
Epoch 6/20
20/20 [==============================] - 0s 6ms/step - loss: 1.4045 - accuracy: 0.4875 - val_loss: 1.6037 - val_accuracy: 0.4187
Epoch 7/20
20/20 [==============================] - 0s 6ms/step - loss: 1.295

/home/ilmari/python/DigitRecognition3D/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
# Do a grid search with Keras tuner
import kerastuner as kt
from tensorflow.keras.regularizers import L1L2

def model_builder(hp):
    """ take in a padded sequence of 3d coordinates, and output a class label
    """
    inputs = tf.keras.Input(shape=(X.shape[1],X.shape[2]))
    
    # Hyperparameters for architecture
    num_conv_layers = hp.Int('num_conv_layers', min_value=1, max_value=4, step=1)
    num_filters = hp.Int('num_filters', min_value=16, max_value=128, step=16)
    # Also define the strenght of regularization
    reg = hp.Choice('reg', values=[0.0, 0.001, 0.01, 0.1])
    x = inputs
    
    for i in range(num_conv_layers):
        x = tf.keras.layers.Conv1D(num_filters, 3, activation='relu', kernel_regularizer=L1L2(l1=reg, l2=reg))(x)
        num_filters *= 2
    x = tf.keras.layers.MaxPooling1D(3)(x)
    

    
    num_conv2_layers = hp.Int('num_conv2_layers', min_value=1, max_value=4, step=1)
    num_conv2_filters = hp.Int('num_conv2_filters', min_value=16, max_value=128, step=16)
    reg2 = hp.Choice('reg2', values=[0.0, 0.001, 0.01, 0.1])
    
    for i in range(num_conv2_layers):
        x = tf.keras.layers.Conv1D(num_conv2_filters, 3, activation='relu', kernel_regularizer=L1L2(l1=reg2, l2=reg2))(x)
        num_conv2_filters *= 2
    
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(10, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, x)
    
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy']
                )
    return model

tuner = kt.Hyperband(model_builder,
                        objective='val_loss',
                        max_epochs=200,
                        factor=6,
                        directory='keras-tuner-logs',
                        project_name='digit_recognition')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
tuner.search(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[early_stop])

print(f"Best hyperparameters: {tuner.get_best_hyperparameters()[0].values}")

best_model = tuner.get_best_models()[0]
best_model.save('best_model.h5')


    
    
    
    



Trial 254 Complete [00h 00m 25s]
val_accuracy: 0.7149999737739563

Best val_accuracy So Far: 33.09809875488281
Total elapsed time: 00h 48m 37s
Best hyperparameters: {'num_conv_layers': 4, 'num_filters': 96, 'reg': 0.1, 'num_conv2_layers': 4, 'num_conv2_filters': 96, 'reg2': 0.0, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}


In [26]:
# Check each tuned result from keras-tuner-logs
import os
import json

# Get all the log directories
log_dirs = os.listdir('keras-tuner-logs/digit_recognition/')

jsons = []
val_losses = []
val_accuracies = []
# Check the accuracy and loss for each trial.json file (one in each log directory)
nums = [0,0,0,0]
for log_dir in log_dirs:
    if not os.path.isdir(f'keras-tuner-logs/digit_recognition/{log_dir}'):
        continue
    with open(f'keras-tuner-logs/digit_recognition/{log_dir}/trial.json') as f:
        trial_json = json.load(f)
    acc = trial_json['metrics']["metrics"]["val_accuracy"]["observations"][0]["value"][0]
    loss = trial_json['metrics']["metrics"]["val_loss"]["observations"][0]["value"][0]
    print(acc)
    
    val_losses.append(loss)
    val_accuracies.append(acc)
    jsons.append(trial_json)
    
    # Update the nums
    num = int("".join([str(i) for i in nums]))
    num += 1
    nums = [int(i) for i in str(num)]

# Find the json with the lowest loss
min_loss = min(val_losses)
min_loss_idx = val_losses.index(min_loss)
print(f"Lowest loss: {min_loss}")
print(f"Params: {jsons[min_loss_idx]}")



0.47999998927116394
0.14000000059604645
0.1550000011920929
0.47999998927116394
0.9599999785423279
0.10499999672174454
0.054999999701976776
0.22499999403953552
0.23999999463558197
0.23499999940395355
0.4449999928474426
0.8349999785423279
0.925000011920929
0.9150000214576721
0.3199999928474426
0.9449999928474426
0.625
0.9100000262260437
0.574999988079071
0.12999999523162842
0.9900000095367432
0.19499999284744263
0.10999999940395355
0.925000011920929
0.9100000262260437
0.5400000214576721
0.9750000238418579
0.125
0.18000000715255737
0.26499998569488525
0.8100000023841858
0.7549999952316284
0.550000011920929
0.26499998569488525
0.17499999701976776
0.5699999928474426
0.3100000023841858
0.10000000149011612
0.18000000715255737
0.125
0.9399999976158142
0.15000000596046448
0.11999999731779099
0.07000000029802322
0.13500000536441803
0.23499999940395355
0.22499999403953552
0.20999999344348907
0.13500000536441803
0.11500000208616257
0.7099999785423279
0.20999999344348907
0.9300000071525574
0.865000